In [7]:
from WebTable import findTables, pullTable
# from TeamRenamer import Abv2Full, Full2Abv
from bs4 import BeautifulSoup as BS
from bs4 import Comment
from gazpacho import get, Soup
import pandas as pd
import re
from tqdm import tnrange, tqdm_notebook
from time import sleep
import warnings
warnings.filterwarnings('ignore')

In [9]:
def Abv2Full(team_s):
    try:
        if team_s == 'crd' or team_s == 'ari':
            ts = 'Arizona Cardinals'
        elif team_s == 'atl':
            ts = 'Atlanta Falcons'
        elif team_s == 'rav' or team_s == 'bal':
            ts = 'Baltimore Ravens'
        elif team_s == 'buf':
            ts = 'Buffalo Bills'
        elif team_s == 'car':
            ts = 'Carolina Panthers'
        elif team_s == 'chi':
            ts = 'Chicago Bears'
        elif team_s == 'cin':
            ts = 'Cincinnati Bengals'
        elif team_s == 'cle':
            ts = 'Cleveland Browns'
        elif team_s == 'dal':
            ts = 'Dallas Cowboys'
        elif team_s == 'den':
            ts = 'Denver Broncos'
        elif team_s == 'det':
            ts = 'Detroit Lions'
        elif team_s == 'gnb':
            ts = 'Green Bay Packers'
        elif team_s == 'htx' or team_s == 'hou':
            ts = 'Houston Texans'
        elif team_s == 'clt' or team_s == 'ind':
            ts = 'Indianapolis Colts'
        elif team_s == 'jax':
            ts = 'Jacksonville Jaguars'
        elif team_s == 'kan':
            ts = 'Kansas City Chiefs'
        elif team_s == 'lac':
            ts = 'LA Chargers'
        elif team_s == 'lar':
            ts = 'LA Rams'
        elif team_s == 'mia':
            ts = 'Miami Dolphins'
        elif team_s == 'min':
            ts = 'Minnesota Vikings'
        elif team_s == 'nwe':
            ts = 'New England Patriots'
        elif team_s == 'nor':
            ts = 'New Orleans Saints'
        elif team_s == 'nyg':
            ts = 'New York Giants'
        elif team_s == 'nyj':
            ts = 'New York Jets'
        elif team_s == 'rai' or team_s == 'oak':
            ts = 'Oakland Raiders'
        elif team_s == 'phi':
            ts = 'Philadelphia Eagles'
        elif team_s == 'pit':
            ts = 'Pittsburgh Steelers'
        elif team_s == 'sdg':
            ts = 'San Diego Chargers'
        elif team_s == 'sfo':
            ts = 'San Francisco 49ers'
        elif team_s == 'sea':
            ts = 'Seattle Seahawks'
        elif team_s == 'ram' or team_s == 'stl':
            ts = 'St. Louis Rams'
        elif team_s == 'tam':
            ts = 'Tampa Bay Buccaneers'
        elif team_s == 'oti' or team_s == 'ten':
            ts = 'Tennessee Titans'
        elif team_s == 'was':
            ts = 'Washington Redskins'
    except NameError:
        print('Invalid team name')
    return ts

def Full2Abv(team_s):
    try:
        if team_s == 'Arizona Cardinals':
            ts = 'crd'
        elif team_s == 'Atlanta Falcons':
            ts = 'atl'
        elif team_s == 'Baltimore Ravens':
            ts = 'rav'
        elif team_s == 'Buffalo Bills':
            ts = 'buf'
        elif team_s == 'Carolina Panthers':
            ts = 'car'
        elif team_s == 'Chicago Bears':
            ts = 'chi'
        elif team_s == 'Cincinnati Bengals':
            ts = 'cin'
        elif team_s == 'Cleveland Browns':
            ts = 'cle'
        elif team_s == 'Dallas Cowboys':
            ts = 'dal'
        elif team_s == 'Denver Broncos':
            ts = 'den'
        elif team_s == 'Detroit Lions':
            ts = 'det'
        elif team_s == 'Green Bay Packers':
            ts = 'gnb'
        elif team_s == 'Houston Texans':
            ts = 'htx'
        elif team_s == 'Indianapolis Colts':
            ts = 'clt'
        elif team_s == 'Jacksonville Jaguars':
            ts = 'jax'
        elif team_s == 'Kansas City Chiefs':
            ts = 'kan'
        elif team_s == 'Miami Dolphins':
            ts = 'mia'
        elif team_s == 'Minnesota Vikings':
            ts = 'min'
        elif team_s == 'New England Patriots':
            ts = 'nwe'
        elif team_s == 'New Orleans Saints':
            ts = 'nor'
        elif team_s == 'New York Giants':
            ts = 'nyg'
        elif team_s == 'New York Jets':
            ts = 'nyj'
        elif team_s == 'Oakland Raiders':
            ts = 'rai'
        elif team_s == 'Philadelphia Eagles':
            ts = 'phi'
        elif team_s == 'Pittsburgh Steelers':
            ts = 'pit'
        elif team_s == 'San Diego Chargers':
            ts = 'sdg'
        elif team_s == 'San Francisco 49ers':
            ts = 'sfo'
        elif team_s == 'Seattle Seahawks':
            ts = 'sea'
        elif team_s == 'St. Louis Rams':
            ts = 'ram'
        elif team_s == 'Tampa Bay Buccaneers':
            ts = 'tam'
        elif team_s == 'Tennessee Titans':
            ts = 'oti'
        elif team_s == 'Washington Redskins':
            ts = 'was'
    except NameError:
        print('Invalid team name')
    return ts

In [ ]:
Abv2Full('stl')
# Full2Abv('Arizona Cardinals')

In [ ]:
team_list = ['crd', 'atl', 'rav', 'buf', 'car', 'chi', 'cin', 'cle', 'dal', 'den', 'det','gnb', 
'htx', 'clt', 'jax', 'kan', 'mia', 'min', 'nwe', 'nor', 'nyg', 'nyj', 'rai', 'phi', 'pit', 
'sdg', 'sfo', 'sea', 'ram', 'tam', 'oti', 'was']

In [ ]:
def urlSplit(inputlist):
    ''' Used to slice the boxscore link to get and sort link and date
    '''
    urlList = []
    for item in inputlist:
        sort = item.split('/')[-1][:9]
        urlList.append(sort)
    return sorted(urlList, reverse=False)

def Nanreplace(x):
    if pd.isnull(x['relative_humidity']):
        return x['relative humidity']
    else:
        return x['relative_humidity']

def boxscore(url):
    '''returns the link to the boxscore and the date the game was played'''
    html = get(url)
    soup = Soup(html)
    boxlinks = str(soup.find('a', {'href': 'boxscore'})).split('"')                    # Finds all a tags with a href link and splits on "
    links_list = [x for x in boxlinks if x.startswith('/boxscore')][1:-3]              # creates a list of all the links that starts with /boxscore
    links_list = ['https://www.pro-football-reference.com' + x for x in links_list]    # creates the complete url to the boxscore links
    df = pd.DataFrame(links_list, columns =['Links'])                                  # creates a dataframe containing all of the url links just created
    df['Date'] = urlSplit(links_list)                                                  # add a new column that uses urlSplit to get an idea of the date
    return df

def get_boxscore_stats(url):
    '''reutns a dataframe that contains home and away boxscores
    use the link from the boxscore link generated from the master
    dataframe
    '''
    boxstats = pullTable(url,'team_stats')                                             # Pull box table from boxscore url
    boxstats.columns = ['Stats', 'Away', 'Home']                                       # rename column names
    homebox = boxstats[['Stats','Home']]                                               # create a list for home and away teams
    awaybox = boxstats[['Stats','Away']]
    homebox = homebox.transpose()                                                      # Transpose the tables
    awaybox = awaybox.transpose()
    home_colnames = ['Home_' + x for x in homebox.iloc[0]]                             # add Home or Away to column names
    away_colnames = ['Away_' + x for x in awaybox.iloc[0]]
    zip_Homeboxscore = list(zip(home_colnames, homebox.iloc[1]))                       # zip list of columns and data together
    zip_Awayboxscore = list(zip(away_colnames, awaybox.iloc[1]))
    boxscorelist = zip_Homeboxscore + zip_Awayboxscore                                 # combine lists together
    box_col = [col[0].replace(' ','_').replace('.','') for col in boxscorelist]        # Replace spaces with _ and remove .
    box_stats = pd.Series([col[1] for col in boxscorelist], index=box_col)             
    boxdf= pd.DataFrame(columns=box_col)
    boxdf = boxdf.append(box_stats, ignore_index=True)
    #Split multiple data from one columns into individual columns
    statSplit = [x.split('-') for x in boxdf['Home_Rush-Yds-TDs']]
    boxdf['Home_Rush_Att'] = statSplit[0][0]
    boxdf['Home_Rush_Yds'] = statSplit[0][1]
    boxdf['Home_Rush_TDs'] = statSplit[0][2]
    statSplit = [x.split('-') for x in boxdf['Away_Rush-Yds-TDs']]
    boxdf['Away_Rush_Att'] = statSplit[0][0]
    boxdf['Away_Rush_Yds'] = statSplit[0][1]
    boxdf['Away_Rush_TDs'] = statSplit[0][2]
    statSplit = [x.split('-') for x in boxdf['Home_Cmp-Att-Yd-TD-INT']]
    boxdf['Home_Pass_Comp'] = statSplit[0][0]
    boxdf['Home_Pass_Att'] = statSplit[0][1]
    boxdf['Home_Pass_Yd'] = statSplit[0][2]
    boxdf['Home_Pass_TD'] = statSplit[0][3]
    boxdf['Home_Pass_INT'] = statSplit[0][4]
    statSplit = [x.split('-') for x in boxdf['Away_Cmp-Att-Yd-TD-INT']]
    boxdf['Away_Pass_Comp'] = statSplit[0][0]
    boxdf['Away_Pass_Att'] = statSplit[0][1]
    boxdf['Away_Pass_Yd'] = statSplit[0][2]
    boxdf['Away_Pass_TD'] = statSplit[0][3]
    boxdf['Away_Pass_INT'] = statSplit[0][4]
    statSplit = [x.split('-') for x in boxdf['Home_Penalties-Yards']]
    boxdf['Home_Penalties'] = statSplit[0][0]
    boxdf['Home_Penalties_Yd'] = statSplit[0][1]
    statSplit = [x.split('-') for x in boxdf['Away_Penalties-Yards']]
    boxdf['Away_Penalties'] = statSplit[0][0]
    boxdf['Away_Penalties_Yd'] = statSplit[0][1]
    statSplit = [x.split('-') for x in boxdf['Home_Sacked-Yards']]
    boxdf['Home_Sacks'] = statSplit[0][0]
    boxdf['Home_SackedYards'] = statSplit[0][1]
    statSplit = [x.split('-') for x in boxdf['Away_Sacked-Yards']]
    boxdf['Home_Sacks'] = statSplit[0][0]
    boxdf['Home_SackedYards'] = statSplit[0][1]
    statSplit = [x.split('-') for x in boxdf['Home_Fumbles-Lost']]
    boxdf['Home_Fumbles'] = statSplit[0][0]
    boxdf['Home_Fumbles_Losts'] = statSplit[0][1]
    statSplit = [x.split('-') for x in boxdf['Away_Fumbles-Lost']]
    boxdf['Away_Fumbles'] = statSplit[0][0]
    boxdf['Away_Fumbles_Losts'] = statSplit[0][1]
    #Conversion to precentages 
    statSplit = [x.split('-') for x in boxdf['Home_Third_Down_Conv']]
    if int(statSplit[0][1]) != 0: 
        boxdf['Home_Third_Down_Conv'] = int(statSplit[0][0])/int(statSplit[0][1]) * 100
    else: 
        boxdf['Home_Third_Down_Conv'] = 0
    statSplit = [x.split('-') for x in boxdf['Away_Third_Down_Conv']]
    if int(statSplit[0][1]) != 0:
        boxdf['Away_Third_Down_Conv'] = int(statSplit[0][0])/int(statSplit[0][1]) * 100
    else: 
        boxdf['Away_Third_Down_Conv'] = 0
    statSplit = [x.split('-') for x in boxdf['Home_Fourth_Down_Conv']]
    if int(statSplit[0][1]) != 0:
        boxdf['Home_Fourth_Down_Conv'] = int(statSplit[0][0])/int(statSplit[0][1]) * 100
    else:
        boxdf['Home_Fourth_Down_Conv'] = 0 
    statSplit = [x.split('-') for x in boxdf['Away_Fourth_Down_Conv']]
    if int(statSplit[0][1]) != 0:  
        boxdf['Away_Fourth_Down_Conv'] = int(statSplit[0][0])/int(statSplit[0][1]) * 100
    else:
        boxdf['Away_Fourth_Down_Conv'] = 0
    boxdf.drop(['Home_Rush-Yds-TDs','Away_Rush-Yds-TDs','Home_Cmp-Att-Yd-TD-INT','Away_Cmp-Att-Yd-TD-INT',
               'Home_Penalties-Yards','Away_Penalties-Yards','Home_Sacked-Yards','Away_Sacked-Yards',
               'Home_Fumbles-Lost','Away_Fumbles-Lost'], axis=1, inplace=True)
    boxdf['GetBoxScoreURL'] = url
    return boxdf




def getGameInfo(url):
    '''Gets table from Game Info table that is within a HTML comment
    Will return a list of lists were [n][0] = column name and [n][1] is the infomation
    [('Won_Toss', 'Cardinals (deferred)'),
     ('Roof', 'retractable roof (closed)'),
     ('Surface', 'grass'),
     ('Duration', '2:56'),
     ('Attendance', ' 60,104 '),
     ('Vegas Line', 'Arizona Cardinals -2.5'),
     ('Over/Under', '46.0(over) ')
    '''
    html = get(url)
    soup = BS(html, 'html.parser')
    comments = soup.find_all(string=lambda text: isinstance(text, Comment))                        # retrieve all the comments from the HTML
    game_comments = []

    for line in comments:
        if '<div class="table_outer_container">' in line:                                          # loop throught the table i want
            game_comments.append(re.sub('<[^>]+>', ' ', line))                                     # use regex to replace everything between the <> tags and append to game comments
            if '</table>' in line:                                                                 # break out of the for loop once it hits the ending table tag 
                break
    game_info = [x.split('\n') for x in game_comments]                                             # split on newlines to create a clean list
    game_info = game_info[0][5:13]                                                                 # more cleaning, i only want these 8 items
    game_info = [x.replace('Won Toss','Won_Toss') if 'Won Toss' in x else x for x in game_info]    # replace the space in the Won Toss item, cleaning/prep
    game_info                                                                               
    for count, row in enumerate(game_info):
        if count == 5 and 'Weather' in row:                                                        
            game_info.pop(count)
    clean_info = []
    for x in game_info:
        if x[:2] == '  ':
            if x[-2:] == '  ':
                clean_info.append(x[2:-2])
            else:
                clean_info.append(x[2:])
        else:
            clean_info.append(x)
    clean_info = [x.replace('  (','(').replace(' (','(').replace('   ','  ').replace(') ',')') for x in clean_info]
    clean_info = [x for x in clean_info if x]
    game = [x.split('  ') for x in clean_info]
    game_titles = [x[0] for x in game]
    game_data = [x[1] for x in game]
    #split on | and setting max split to 2 and then removing the empty list items 
    game = pd.DataFrame([game_data], columns = game_titles)
    game['VegasLink'] = url
    return game

def getQBStats(url):
    '''Gets Advanced Passing table from the boxscore page.
    The website only has this table in 2018 and 2019'''
    qb = pullTable(url, 'passing_advanced')
    home_col = ['HOME_QB_' + x.replace('/','-') for x in qb.columns]
    away_col = ['AWAY_QB_' + x.replace('/','-') for x in qb.columns]
    qb_cols = home_col + away_col
    awayqb = [x for x in qb.iloc[0]]
    homeqb = [x for x in qb.iloc[1]]
    qblist = homeqb + awayqb
    qbdf = pd.DataFrame([qblist], columns=qb_cols)
    qbdf['HOME_QB_Tm'][0],qbdf['AWAY_QB_Tm'][0] = qbdf['HOME_QB_Tm'][0].lower(),qbdf['AWAY_QB_Tm'][0].lower()
    qbdf['QBLINK'] = url
    return qbdf

def PlayerStats(url):
    '''Will return a single row dataframe with QB, RB1, RB2, WR1, WR2, WR3
    Stats from the provided url. URL must be the boxscore link from the 
    GetMaster Function'''
    html = get(url)
    soup = Soup(html)
    dfstats = pd.read_html(str(soup.find('table')[2]))[0]
    #flatten multi Index header
    columns = [row[0] + '_' + row[1] for row in dfstats.columns]
    columns = [x.replace('Unnamed: 0_level_0_Player', 'Player').replace('Unnamed: 1_level_0_Tm','Team') for x in columns]
    # droping 'Fumble columns'
    columns = [i for i in columns if 'Fumble' not in i]
    home_columns = ['Home_QB_' + x for x in columns]
    away_columns = ['Away_QB_' + x for x in columns]
    if len(dfstats.columns) == 22:
        dfstats.drop(('Fumbles', 'Fmb'), axis = 1, inplace = True)
        dfstats.drop(('Fumbles', 'FL'), axis = 1, inplace = True)
    dfstats.columns = columns
    home_qb = []
    away_qb = []
    qb_cols = away_columns + home_columns
    Home_staters = pullTable(url,'home_starters')
    Away_staters = pullTable(url,'vis_starters')
    Home_staters = Home_staters[:11]
    Away_staters = Away_staters[:11]
    Starters = pd.concat([Home_staters,Away_staters])
    # Joining tables and cleaning table
    positions = dfstats.join(Starters.set_index('Player'), on='Player')
    away_team = positions['Team'].iloc[0]
    home_team = positions['Team'].iloc[-1]
    alinks = str(soup.find('table',{'id':"player_offense"}))
    basenamelink = 'https://www.pro-football-reference.com/'
    namelink = alinks.split('href')
    namelink = namelink[1:]
    namelink = [x.split('"')[1] for x in namelink]
    PosName = []
    Play_Name= []
    for x in namelink:
        getname = basenamelink + x
        html_name = get(getname)
        soup_name = Soup(html_name)
        playerposition = str(soup_name.find('div',{'id':"meta"}))
        playerposition = playerposition.split('Position')
        Pos_Name = playerposition[1][11:13]
        PlayerName = re.sub('<[^>]+>', ' ',playerposition[0]).strip()
        PlayerName = [' '.join(PlayerName.split(' ')[:2])]
        Play_Name.extend(PlayerName)
        PosName.append(Pos_Name)

    NamePosDict = {Play_Name[i] : PosName[i] for i in range(len(Play_Name))}
    positions['Pos']=positions['Player'].apply(lambda x: NamePosDict.get(x,x))
    away_qb = []
    home_qb = []
    for index, row in positions.iterrows():
        if row['Pos'] == 'QB':
            if row['Team'] == home_team:
                home_qb.extend(row)
            else:
                away_qb.extend(row)
    qb = home_qb + away_qb
    qb_filler = ['Bobby Boucher','SCLSU','9999','9999','9999','9999','9999','9999','9999','9999','9999',
                 '9999','9999','9999','9999','9999','9999','9999','9999','9999']
    away_qb = away_qb[:21]
    home_qb = home_qb[:21]
    if away_qb == []:
        away_qb = qb_filler
    if home_qb == []:
        home_qb = qb_filler
    qb = home_qb + away_qb
    qb = [x for x in qb if x!= 'QB']
    qb_col = home_columns + away_columns
    QB = pd.DataFrame([qb], columns=qb_col)
    QB = QB.drop(QB.filter(regex='Receiving').columns, axis=1)

    #Start Cleaning up the postions dataframe and then create RB, WR, TE DataFrames then add them all together to return
    position_drop = ['Passing_Cmp','Passing_Att','Passing_Yds','Passing_TD', 'Passing_Int', 'Passing_Sk',
                    'Passing_Yds.1','Passing_Lng', 'Passing_Rate']
    positions.drop(position_drop, axis = 1, inplace=True)
    positions = positions[positions.Pos != 'QB']
    positions = positions[positions.Team.notna()]
    pos_col = positions.columns
    # positions = positions[(positions.Pos != 'QB')|(positions.Pos.notna())]
    positions = positions[(positions.Pos != 'QB')]

    home_wr = []
    away_wr = []
    home_rb = []
    away_rb = []
    for index, row in positions.iterrows():
        if row['Pos'] == 'RB' or row['Pos'] == 'FB':
            if row['Team'] == home_team:
                home_rb.append(row.values)
            else:
                away_rb.append(row.values)
        elif row['Pos'] == 'WR' or row['Pos'] == 'TE':
            if row['Team'] == home_team:
                home_wr.append(row.values)
            else:
                away_wr.append(row.values)
    #Creating Home and Away DataFrames to split each row out from
    HOMEWRtemp,AWAYWRtemp = pd.DataFrame(home_wr, columns=pos_col),pd.DataFrame(away_wr, columns=pos_col)
    HOMERBtemp,AWAYRBtemp = pd.DataFrame(home_rb, columns=pos_col),pd.DataFrame(away_rb, columns=pos_col)
    #creating column names for each position WR1-3 and RB-1
    hrb1_col, hrb2_col, arb1_col, arb2_col = ['H_RB1_'+ x for x in pos_col],['H_RB2_'+ x for x in pos_col],['A_RB1_'+ x for x in pos_col],['A_RB2_'+ x for x in pos_col]
    hwr1_col, hwr2_col, hwr3_col = ['H_WR1_'+ x for x in pos_col],['H_WR2_'+ x for x in pos_col],['H_WR3_'+ x for x in pos_col]
    awr1_col, awr2_col, awr3_col = ['A_WR1_'+ x for x in pos_col],['A_WR2_'+ x for x in pos_col],['A_WR3_'+ x for x in pos_col]
    #creating a datafram for each play that will be combined into one row later
    hrb_col,arb_col = hrb1_col+hrb2_col,arb1_col+arb2_col
    hwr_col,awr_col = hwr1_col+hwr2_col+hwr3_col, awr1_col+awr2_col+awr3_col
    
    Filler_Rec = ['Forrest Gump', 'Ala', '999', '999', '999', '999', '999', '999', '999', '999',
       '-1', 'XX']
    if HOMERBtemp.shape[0]==1:
        HRBlist = list(HOMERBtemp.iloc[0].values) + Filler_Rec
    else:
        HRBlist = list(HOMERBtemp.iloc[0].values) + list(HOMERBtemp.iloc[1].values)

    if AWAYRBtemp.shape[0]==1:
        ARBlist = list(AWAYRBtemp.iloc[0].values) + Filler_Rec
    else:
        ARBlist = list(AWAYRBtemp.iloc[0].values) + list(AWAYRBtemp.iloc[1].values)

    if HOMEWRtemp.shape[0]==1:
        HWRlist = list(HOMEWRtemp.iloc[0].values) + Filler_Rec + Filler_Rec
    elif HOMEWRtemp.shape[0]==2:
        HWRlist = list(HOMEWRtemp.iloc[0].values) + list(HOMEWRtemp.iloc[1].values) + Filler_Rec
    else:
        HWRlist = list(HOMEWRtemp.iloc[0].values) + list(HOMEWRtemp.iloc[1].values) + list(HOMEWRtemp.iloc[2].values) 

    if AWAYWRtemp.shape[0]==1:
        AWRlist = list(AWAYWRtemp.iloc[0].values) + list(Filler_Rec) + list(Filler_Rec)
    elif AWAYWRtemp.shape[0]==2:
        AWRlist = list(AWAYWRtemp.iloc[0].values) + list(AWAYWRtemp.iloc[1].values) + list(Filler_Rec)
    else:
        AWRlist = list(AWAYWRtemp.iloc[0].values) + list(AWAYWRtemp.iloc[1].values) + list(AWAYWRtemp.iloc[2].values) 

    #Create Home and away Dataframes
    HRB, ARB = pd.DataFrame([HRBlist], columns=hrb_col),pd.DataFrame([ARBlist], columns=arb_col)
    HWR, AWR = pd.DataFrame([HWRlist], columns=hwr_col),pd.DataFrame([AWRlist], columns=awr_col)
    #Concat players stats into one row dataframe
    Player_stats = pd.concat([QB,HRB,ARB,HWR,AWR], join='outer', axis=1)
    Player_stats['PlayersURL'] = url 
    return Player_stats


def get_master_df(teamList, startyear, endyear):
    endyear = endyear + 1
#     years = [x for x in range(startyear,endyear)]
    masterdf = pd.DataFrame()
    for x in tnrange(startyear,endyear, desc='YEAR'):
        nflDF = pd.DataFrame()
        for team in tqdm_notebook((team_list), desc='TEAMS'):
            try:
                year = x
                base = 'https://www.pro-football-reference.com'
                url = base + '/teams/' + team + '/' + str(year) + '.htm'
                html = get(url)
                soup = Soup(html)
                df = pd.read_html(str(soup.find('table')[1]))[0]
                df = df[df['Unnamed: 9_level_0','Opp'] != 'Bye Week']
                df = df[df['Unnamed: 9_level_0','Opp'] != 'Playoffs']
                df['Team','Name'] = Abv2Full(team)
                df['Team','Year'] = year
                top_level = df.columns.get_level_values(0).astype(str)
                lower_level = df.columns.get_level_values(1).astype(str)
                df.columns = top_level + '_' + lower_level
                df.columns = [x.replace("', '","_").replace('Offense','Home').replace('Defense','Away') for x in df.columns]
                df.rename(columns = {'Unnamed: 0_level_0_Week':'Week','Unnamed: 1_level_0_Day':'Day','Unnamed: 2_level_0_Date':'Date',
                                          'Unnamed: 3_level_0_Unnamed: 3_level_1':'Time_ET','Unnamed: 4_level_0_Unnamed: 4_level_1':'BOXSCORE',
                                          'Unnamed: 5_level_0_Unnamed: 5_level_1':'W_L','Unnamed: 6_level_0_OT':'OT','Unnamed: 7_level_0_Rec':'Home_Record',
                                          'Unnamed: 8_level_0_Unnamed: 8_level_1':'HomeOrAway','Unnamed: 9_level_0_Opp':'Opp','Score_Tm':'Home_Score',
                                          'Score_Opp':'Away_Score'},inplace=True)
                df.drop(['Home_1stD','Home_PassY','Home_RushY','Home_TO','Away_1stD','Away_PassY','Away_TO'], axis=1, inplace=True)
                df = df.reset_index(drop=True)
                boxscorelink = boxscore(url)
                mergedf =  df.merge(boxscorelink, left_index=True, right_index=True)
                gameinfo =  pd.DataFrame()
                for index, row in mergedf.iterrows():
                    try:
        #                 Advance Passing stats one good in 2018 and 2019
        #                 qb = getQBStats(row['Links'])
                        qbStats = PlayerStats(row['Links'])
                        boxScoreStats = get_boxscore_stats(row['Links'])
                        vegas = getGameInfo(row['Links'])
        #                 row_info = pd.concat([qb, stats], axis=1, join='inner')
        #                 vegas_stats = pd.concat([row_info, vegas], axis=1, join='inner')
        #                 vegas_stats = stats.join(vegas.set_index('VegasLink'), on='GetBoxScoreURL')
                        gamerow = pd.concat([qbStats,boxScoreStats,vegas], join='outer',axis=1)
                        gameinfo = gameinfo.append(gamerow, ignore_index=True, sort=False)
                    except:
                        pass
                gameinfo = pd.merge(mergedf, gameinfo, left_on='Links', right_on='GetBoxScoreURL')
                if 'Date_x' in gameinfo.columns:
                    gameinfo.rename(columns = {'Date_x':'Date'}, inplace=True)
                nflDF = nflDF.append(gameinfo,ignore_index=True, sort = False)
            except:
                pass
        masterdf = masterdf.append(nflDF,ignore_index=True, sort = False)
        sleep(1)
    return masterdf

In [ ]:
master = get_master_df(team_list,2010,2020)
master.to_csv(r'./data/MASTER2010_2020.csv', encoding='utf-8', header='true')

In [ ]:
master = get_master_df(team_list,2002,2010)
master.to_csv(r'./data/MASTER2002_2010.csv', encoding='utf-8', header='true')

In [4]:
master02_10 = pd.read_csv('./data/MASTER2002_2010.csv',index_col='Unnamed: 0')
master10_20 = pd.read_csv('./data/MASTER2010_2020.csv',index_col='Unnamed: 0')

In [72]:
master = pd.concat([master02_10,master10_20])
master['HomeOrAway'] = master['HomeOrAway'].astype(str)
master = master[~master.HomeOrAway.str.contains('@')]
master.drop(['PlayersURL','VegasLink','GetBoxScoreURL','Weather','BOXSCORE', 'HomeOrAway','Links'], axis=1, inplace=True)
master = master.reset_index(drop=True)

In [11]:
master.to_csv(r'./data/MASTERFULL.csv', encoding='utf-8', header='true')